In [36]:
import pandas as pd
from scipy.stats import mannwhitneyu
import numpy as np

In [5]:
t = pd.read_csv('2022-09-21 17-47-27.csv', sep = ';', encoding = 'cp1251')
t.head()

,num,name_rus,rating_new,origin,genre,rating_old,qty_views
0,1,Зеленая миля,9.1,США,фэнтези/ драма,8.9,692418
1,2,Побег из Шоушенка,9.1,США,драма,8.9,784326
2,3,Властелин колец: Возвращение короля,8.6,Новая Зеландия/ США,фэнтези/ приключения,8.8,481829
3,4,Властелин колец: Две крепости,8.6,Новая Зеландия/ США,фэнтези/ приключения,8.8,467607
4,5,Властелин колец: Братство Кольца,8.6,Новая Зеландия/ США,фэнтези/ приключения,8.8,516856


In [10]:
t.describe()

,num,rating_new,rating_old,qty_views
count,250.000000,250.000000,250.000000,2.500000e+02
mean,125.500000,8.184400,8.179600,2.893783e+05
std,72.312977,0.272643,0.193114,1.868284e+05
min,1.000000,7.600000,8.000000,2.005600e+04
25%,63.250000,8.000000,8.000000,1.450212e+05
50%,125.500000,8.100000,8.100000,2.516295e+05
75%,187.750000,8.300000,8.300000,4.053790e+05
max,250.000000,9.100000,8.900000,1.303016e+06


From description statistics it seems as though ratings are almost the same over time: similar mean, median and std. Let's compare the new and old ratings overall with the Mann-Whitney U test.

In [8]:
u1, p = mannwhitneyu(x = t.rating_new, y = t.rating_old, alternative = 'two-sided')
u1, p

(31175.5, 0.9629567921262221)

Indeed, there is no statistial difference in ratings overall: the p-value is too big. Let's compare data by genre/country:

In [15]:
t.groupby('origin').count().sort_values(by = 'num', ascending = False).head()

,num,name_rus,rating_new,genre,rating_old,qty_views
origin,,,,,,
США,110,110,110,110,110,110
СССР,31,31,31,31,31,31
Великобритания/ США,17,17,17,17,17,17
США/ Германия,10,10,10,10,10,10
Япония,9,9,9,9,9,9


Let's look at differences in USA over time and USSR over time.

In [16]:
u1, p = mannwhitneyu(x = t.loc[t.origin == 'США', 'rating_new'], y = t.loc[t.origin == 'США', 'rating_old']
                     , alternative = 'two-sided')
u1, p

(5634.0, 0.3718029670954922)

In [17]:
u1, p = mannwhitneyu(x = t.loc[t.origin == 'СССР', 'rating_new'], y = t.loc[t.origin == 'СССР', 'rating_old']
                     , alternative = 'two-sided')
u1, p

(669.0, 0.007358118927667391)

In [19]:
t.loc[t.origin == 'СССР', 'rating_new'].shape

(31,)

For USA there is no statistically significant difference over time, but for USSR there is. Despite the small sample size, Mann-Whitney U test should be still suited to detect it. Let's look at genres 

In [21]:
t.groupby('genre').count().sort_values(by = 'num', ascending = False).head()

,num,name_rus,rating_new,origin,rating_old,qty_views
genre,,,,,,
фантастика/ боевик,19,19,19,19,19,19
мультфильм/ фэнтези,13,13,13,13,13,13
драма/ мелодрама,13,13,13,13,13,13
драма,12,12,12,12,12,12
триллер/ драма,11,11,11,11,11,11


In [22]:
u1, p = mannwhitneyu(x = t.loc[t.genre == 'фантастика/ боевик', 'rating_new'], y = t.loc[t.genre == 'фантастика/ боевик', 'rating_old']
                     , alternative = 'two-sided')
u1, p

(121.5, 0.0787049093768963)

There is no statistical difference (or at least a small chance of it) between ratings of action movies over time. Let's check the same thing with bootstrap

In [34]:
from bstrap import bootstrap

In [38]:
old_stats, new_stats, p_value = bootstrap(np.mean, t.rating_old, t.rating_new, nbr_runs=1000)
old_stats, new_stats, p_value

({'avg_metric': 8.179429999999991,
  'metric_ci_lb': 8.160399999999992,
  'metric_ci_ub': 8.19881999999999},
 {'avg_metric': 8.184998399999996,
  'metric_ci_lb': 8.156799999999995,
  'metric_ci_ub': 8.215199999999998},
 0.806)

As we can see, there is still no significant statistical difference between ratings. Let's look at countries and genres:

In [39]:
old_stats, new_stats, p_value = bootstrap(np.mean, t.loc[t.origin == 'США', 'rating_old'], t.loc[t.origin == 'США', 'rating_new'], nbr_runs=1000)
old_stats, new_stats, p_value

({'avg_metric': 8.177637272727278,
  'metric_ci_lb': 8.149090909090916,
  'metric_ci_ub': 8.206363636363644},
 {'avg_metric': 8.15514181818182,
  'metric_ci_lb': 8.111772727272728,
  'metric_ci_ub': 8.20181818181818},
 0.519)

In [40]:
old_stats, new_stats, p_value = bootstrap(np.mean, t.loc[t.origin == 'СССР', 'rating_old'], t.loc[t.origin == 'СССР', 'rating_new'], nbr_runs=1000)
old_stats, new_stats, p_value

({'avg_metric': 8.237703225806452,
  'metric_ci_lb': 8.183870967741935,
  'metric_ci_ub': 8.290483870967744},
 {'avg_metric': 8.378712903225807,
  'metric_ci_lb': 8.325806451612904,
  'metric_ci_ub': 8.43225806451613},
 0.003)

In [41]:
old_stats, new_stats, p_value = bootstrap(np.mean, t.loc[t.genre == 'фантастика/ боевик', 'rating_old'], 
                                          t.loc[t.genre == 'фантастика/ боевик', 'rating_new'], nbr_runs=1000)
old_stats, new_stats, p_value

({'avg_metric': 8.16688947368421,
  'metric_ci_lb': 8.110526315789471,
  'metric_ci_ub': 8.231578947368419},
 {'avg_metric': 8.06256842105263,
  'metric_ci_lb': 7.957894736842104,
  'metric_ci_ub': 8.173684210526313},
 0.142)

As we can see, the results are consistent with what we saw with Mann-Whitney U tests: only USSR ratings are different over time, USA and action movies are not statistically significantly different.
